# Parameter Tuning

In [1]:
!pip install category-encoders
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 11.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=789ea3d380bfdff128f13fa6ee

In [2]:
# Global variables
RANDOM_SEED = 153
NUM_TRIALS = 100

In [3]:
# Imports
import joblib
import time
import numpy as np
import pandas as pd
import gc; gc.enable()

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Speedup some scikit-learn algorithms
#from sklearnex import patch_sklearn
#patch_sklearn()
import sklearn

# Scoring and Cross-Validation
from sklearn.base import clone
from sklearn.metrics import roc_auc_score
from sklearn.utils.extmath import softmax

# Imputation and Preprocessing
from sklearn.preprocessing import RobustScaler
from category_encoders import WOEEncoder
from sklearn.impute import KNNImputer

# Pipeline Constructors
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer

# Models
from sklearn.linear_model import LogisticRegression, HuberRegressor
from xgboost import XGBClassifier

# Optuna
import optuna
from optuna.visualization import plot_param_importances, plot_parallel_coordinate
from optuna.pruners import PercentilePruner

In [8]:
submission = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv', index_col = 'id') 
test = pd.read_csv('test.csv', index_col = 'id')

# Remove target column
target = train['failure'].copy()
gc.collect()

print(target)

id
0        0
1        0
2        0
3        0
4        0
        ..
26565    0
26566    0
26567    0
26568    0
26569    0
Name: failure, Length: 26570, dtype: int64


# Preprocessing

In [9]:
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']# product code?
    feature = [f for f in df_test.columns if f.startswith('measurement') or f =='loading']

    full_fill_dict ={}
    full_fill_dict['measurement_17'] = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    # collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
    a = []
    b =[]
    for x in range(3,17):
        corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
        a.append(np.round(np.sum(corr[1:4]),3)) # add the 3 first lines of the correlation values to get the "most correlated"
        b.append(f'measurement_{x}')
    c = pd.DataFrame()
    c['Selected columns'] = b
    c['correlation total'] = a
    c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)

    #display(c.head(10))

    for i in range(10):
        #print(c.iloc[i,0])
        measurement_col = c.iloc[i,0] #select the next best correlated column 
        fill_dict = {}
        for x in data.product_code.unique(): 
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            #print(corr)
            fill_dict[x] = corr[1:5].index.tolist()
        full_fill_dict[measurement_col] = fill_dict
        #print(full_fill_dict[measurement_col])

    feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]

    for code in data.product_code.unique():#A to I
        for measurement_col in list(full_fill_dict.keys()):
            tmp = data[data.product_code == code]
            column = full_fill_dict[measurement_col][code]
            tmp_train = tmp[column+[measurement_col]].dropna(how='any')
            tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]

            model = HuberRegressor(epsilon=1.9)
            model.fit(tmp_train[column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])

        # others NA columns:
        data.loc[data.product_code==code, feature] = KNNImputer(n_neighbors=3).fit_transform(data.loc[data.product_code==code, feature])

    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]
  
    
    return df_train, df_test

In [10]:
train, test = preprocessing(train, test)
features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'area', 'm3_missing', 'm5_missing', 'measurement_avg']
train.head()



,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,m3_missing,m5_missing,area,measurement_avg
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7.0,8.0,4.0,18.040,...,15.029,15.495868,13.034,14.684,764.100,0.0,0,0,45,15.360919
1,A,84.89,material_7,material_8,9,5,14.0,3.0,3.0,18.213,...,14.732,15.425000,14.395,15.631,682.057,0.0,0,0,45,15.446286
2,A,82.43,material_7,material_8,9,5,12.0,1.0,5.0,18.057,...,16.711,18.631000,14.094,17.946,663.376,0.0,0,0,45,16.098820
3,A,101.07,material_7,material_8,9,5,13.0,2.0,6.0,17.295,...,15.250,15.562000,16.154,17.172,826.282,0.0,0,0,45,15.599500
4,A,188.06,material_7,material_8,9,5,9.0,2.0,8.0,19.346,...,16.182,12.760000,13.153,16.412,579.885,0.0,0,0,45,15.194071


In [15]:
def split(X: pd.DataFrame):
    indices = list(X.groupby("product_code").indices.values())
    for i in range(len(indices)):
        for j in range(i + 1, len(indices)): #4+3+2+1
            yield [
                np.concatenate([ix for k, ix in enumerate(indices) if k not in [i, j]]),#3
                np.concatenate([ix for k, ix in enumerate(indices) if k in [i, j]])#2
            ]

In [16]:

SPLITS = list(split(train))
print(len(SPLITS))
print(SPLITS)

10
[[array([10350, 10351, 10352, ..., 26567, 26568, 26569]), array([    0,     1,     2, ..., 10347, 10348, 10349])], [array([ 5100,  5101,  5102, ..., 26567, 26568, 26569]), array([    0,     1,     2, ..., 16112, 16113, 16114])], [array([ 5100,  5101,  5102, ..., 26567, 26568, 26569]), array([    0,     1,     2, ..., 21224, 21225, 21226])], [array([ 5100,  5101,  5102, ..., 21224, 21225, 21226]), array([    0,     1,     2, ..., 26567, 26568, 26569])], [array([    0,     1,     2, ..., 26567, 26568, 26569]), array([ 5100,  5101,  5102, ..., 16112, 16113, 16114])], [array([    0,     1,     2, ..., 26567, 26568, 26569]), array([ 5100,  5101,  5102, ..., 21224, 21225, 21226])], [array([    0,     1,     2, ..., 21224, 21225, 21226]), array([ 5100,  5101,  5102, ..., 26567, 26568, 26569])], [array([    0,     1,     2, ..., 26567, 26568, 26569]), array([10350, 10351, 10352, ..., 21224, 21225, 21226])], [array([    0,     1,     2, ..., 21224, 21225, 21226]), array([10350, 10351, 10352,

# Scoring Function

In [ ]:
preprocessing = make_pipeline(
    make_column_transformer(
        (WOEEncoder(), ['attribute_0']),
        (FunctionTransformer(np.log1p), ['loading']),
        remainder = 'passthrough'
    ),
    RobustScaler()
)

In [ ]:
# Percentile Pruner settings
pruner = PercentilePruner(
    percentile = 75,
    n_startup_trials = 15,
    n_warmup_steps = 5,
    interval_steps = 1,
    n_min_trials = 15,
)

In [ ]:

# Scoring Function
def Score(sklearn_model, model_save = 0):

    # Stores predictions and scores
    
    scores = np.zeros(len(SPLITS))
    model = None
    # Stratified k-fold cross-validation
    for fold, (train_idx, valid_idx) in enumerate(SPLITS):

        # Training and Validation Sets
        X_train = train[features].iloc[train_idx].copy()
        X_valid = train[features].iloc[valid_idx].copy()
        y_train = target.iloc[train_idx].copy()
        y_valid = target.iloc[valid_idx].copy()

       # Define model and train
        model = make_pipeline(
            clone(preprocessing),
            clone(sklearn_model)
        )
        model.fit(X_train, y_train)
        if model_save:  
          joblib.dump(model,'model'+str(fold)+'.pkl')

        scores[fold] = roc_auc_score(y_valid, model.predict_proba(X_valid)[:,1])

    print(f'Worst: {round(np.min(scores), 6)}')
    print(f'Average:{round(np.mean(scores), 6)}')
    return np.mean(scores)

# XGBoost

In [ ]:
default_params = dict(            
    booster = 'gblinear',
    eval_metric = 'logloss',
    random_state = RANDOM_SEED,
)

In [ ]:
def parameter_search(trials):
    
    # Optuna objective function
    def objective(trial):
        model_params = dict( 
            # default 0
            reg_lambda = trial.suggest_loguniform(
                "reg_lambda", 1e-10, 10
            ),
            # default 0
            reg_alpha = trial.suggest_loguniform(
                "reg_alpha", 1e-10, 10
            ),
            # default cyclic
            feature_selector = trial.suggest_categorical(
                "feature_selector",['cyclic','shuffle']
            ),
        )
        model = XGBClassifier(
            **default_params,
            **model_params
        )
        return Score(model,0)
    
    
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study = optuna.create_study(pruner = pruner,direction = "maximize")
    
    # Close to defaults
    study.enqueue_trial({
        'reg_lambda': 1e-10, 
        'reg_alpha': 1e-10, 
        'feature_selector': 'cyclic'
    })
    study.optimize(objective, n_trials=trials)
    return study

In [ ]:
# Hide output
study = parameter_search(NUM_TRIALS)

[I 2023-01-08 14:58:03,588] A new study created in memory with name: no-name-956227b3-a721-4fde-a0f5-c89b24f7b6ff
[D 2023-01-08 14:58:03,597] Trial 0 popped from the trial queue.
[I 2023-01-08 14:58:09,879] Trial 0 finished with value: 0.5885290209959226 and parameters: {'reg_lambda': 1e-10, 'reg_alpha': 1e-10, 'feature_selector': 'cyclic'}. Best is trial 0 with value: 0.5885290209959226.


Worst: 0.581888
Average:0.588529


[I 2023-01-08 14:58:18,906] Trial 1 finished with value: 0.5885290659189659 and parameters: {'reg_lambda': 1.7224359887850736e-10, 'reg_alpha': 2.0937117085130584e-07, 'feature_selector': 'cyclic'}. Best is trial 1 with value: 0.5885290659189659.


Worst: 0.581889
Average:0.588529


[I 2023-01-08 14:58:27,566] Trial 2 finished with value: 0.5885288408286115 and parameters: {'reg_lambda': 1.4317121770511305e-07, 'reg_alpha': 8.296755262382416e-10, 'feature_selector': 'shuffle'}. Best is trial 1 with value: 0.5885290659189659.


...

Worst: 0.583968
Average:0.589284


[I 2023-01-08 15:06:11,150] Trial 98 finished with value: 0.5892345627789359 and parameters: {'reg_lambda': 0.005920594409961759, 'reg_alpha': 0.0002974337042895817, 'feature_selector': 'shuffle'}. Best is trial 51 with value: 0.5892903927713706.


Worst: 0.584105
Average:0.589235


[I 2023-01-08 15:06:15,841] Trial 99 finished with value: 0.5890130907505325 and parameters: {'reg_lambda': 0.0010758921063307816, 'reg_alpha': 5.338918219482402e-05, 'feature_selector': 'shuffle'}. Best is trial 51 with value: 0.5892903927713706.


Worst: 0.582815
Average:0.589013


In [ ]:
print("Best Parameters:", study.best_params)

Best Parameters: {'reg_lambda': 0.004326989753682215, 'reg_alpha': 3.0800540492811244e-07, 'feature_selector': 'cyclic'}


In [ ]:
Score(XGBClassifier(**default_params, **study.best_params),1)

Worst: 0.583927
Average:0.58929


"model=Score(XGBClassifier(**default_params, **study.best_params))[1]\njoblib.dump(model,'model.pkl')\ntest_preds = np.zeros((test.shape[0],)) \ntest_preds += model.predict_proba(test[features])[:, 1] / len(SPLITS)\nprint(test_preds)"